In [2]:
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import csv
import glob

def Read_Data(WeekNumber, Year):
#	pass
      columns=["Mode", "Date", "DateTime", "CardID", "CardType", "VehicleID",
                   "ParentRoute", "RouteID", "StopID"] 

     # fname = "{0}/{1}/{2}/{3}/".format(path, scan_type, year, week)
      fname = glob.glob(Year + '/Week%s' %WeekNumber + "/*.txt")
      print(fname)
      dataset = pd.read_csv(fname[0], sep="|", names=columns)
   
      # Creating separate columns for Year, Month, Day
      dataset['Date']  = pd.to_datetime(dataset['Date'])
      dataset['Year']  = dataset['Date'].dt.year
      dataset['Month'] = dataset['Date'].dt.month
      dataset['Day']   = dataset['Date'].dt.day

      #Reading in stoplocation data:
      columns2 = ["StopID", "StopName", "Bla", "Bla", "Bla", "PostCode", "City","Bla", "MetroName", "lat", "long"]
      dataset2 = pd.read_csv('stop_locations.txt', sep="|", names = columns2)
      # creating a file with all data and their matched latitued and longitude cooridnates.
      newdata =  dataset2[['StopID', 'StopName','PostCode', 'lat', 'long']]
      
      All = pd.merge(dataset, newdata, how='right', on='StopID')
      return All


def Append_data(Year, FirstWeek, LastWeek):
    basic_table = Read_Data(FirstWeek, Year)
    for i in range(FirstWeek+1, LastWeek+1):
          Actual_Table    = Read_Data(i, Year)
          basic_table     = basic_table.append(Actual_Table) 
          print('appended', len(basic_table))
    return basic_table


dataset_2015 = Append_data("2015", 27, 53)
dataset_2017 = Append_data("2017", 27, 53)


['2015/Week27/file27.txt']


/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


['2015/Week28/file28.txt']
appended 562949
['2015/Week29/file29.txt']
appended 938763
['2015/Week30/file30.txt']
appended 1345681
['2015/Week31/file31.txt']
appended 1756802
['2015/Week32/file32.txt']
appended 2167346
['2015/Week33/file33.txt']
appended 2582381
['2015/Week34/file34.txt']
appended 3000498
['2015/Week35/file35.txt']
appended 3415944
['2015/Week36/file36.txt']
appended 3812335
['2015/Week37/file37.txt']
appended 4227738
['2015/Week38/file38.txt']
appended 4644584
['2015/Week39/file39.txt']
appended 5036225
['2015/Week40/file40.txt']
appended 5386724
['2015/Week41/file41.txt']
appended 5800822
['2015/Week42/file42.txt']
appended 6221885
['2015/Week43/file43.txt']
appended 6638975
['2015/Week44/file44.txt']
appended 7045497
['2015/Week45/file45.txt']
appended 7397961
['2015/Week46/file46.txt']
appended 7804812
['2015/Week47/file47.txt']
appended 8213773
['2015/Week48/file48.txt']
appended 8616071
['2015/Week49/file49.txt']
appended 9017024
['2015/Week50/file50.txt']
appende

In [4]:
def group(mydata):  
    #since groupby gets rid of columns, we are gonna keep the following ones:
    dataset_loc       = mydata[["StopID", "StopName", "PostCode", "lat", "long"]] 
    # groupig the dataset by StopID, and counting the number of touch-ons!!!
    dataset_grouped   = mydata.groupby("StopID").CardID.count().reset_index()

    All                    = pd.merge(dataset_grouped, dataset_loc, how ='left', on = 'StopID')
    All.drop_duplicates(subset = "StopID", keep='first', inplace=True)
    return All

All_2015_grouped = group(dataset_2015)
All_2017_grouped = group(dataset_2017)


In [7]:
import folium 

dataset_2015_2017 = pd.merge(All_2015_grouped , All_2017_grouped , on = "StopID" ) 
# the map will not render if there are too many points
# sort it and take the highest 300
dataset_2015_2017 = dataset_2015_2017.sort_values(by=["CardID_x"]) 
dataset_2015_2017 = dataset_2015_2017.tail(300)

Delta_R = dataset_2015_2017.CardID_y - dataset_2015_2017.CardID_x #)np.abs(

# creating growth and decline datasets
dataset_growth  = dataset_2015_2017[Delta_R > 0]
dataset_decline = dataset_2015_2017[Delta_R < 0]


In [8]:

## growth 
Delta_R_growth      = dataset_growth.CardID_y - dataset_growth.CardID_x
total_number       =  Delta_R_growth.count()
print('total number', total_number)
Delta_R_growth_ratio = Delta_R_growth / np.sqrt(total_number) 


dataset_growth["Delta_R_growth"] = Delta_R_growth_ratio
print(dataset_growth)

m2 = folium.Map(location=[-37.9,145], tiles="openstreetmap",  zoom_start=9.5) #,
 
# I can add marker one by one on the map
for i in range(0,len(dataset_growth)):
   folium.Circle(
      location=[dataset_growth.iloc[i]['lat_x'], dataset_growth.iloc[i]['long_x']],
      popup=dataset_growth.iloc[i]['StopName_x'], 
      radius=dataset_growth.iloc[i]['Delta_R_growth'],
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(m2)
 

total number 208
       StopID  CardID_x                           StopName_x  PostCode_x  \
15331   19491      3857              Lincoln Square - Stop 3      3053.0   
25999   47930      3917                         Chadstone SC      3145.0   
156       187      3934                  Greensborough Plaza      3088.0   
26057   47997      3970                    Melbourne Airport      3045.0   
22857   42273      3974                      Chelsea Station      3196.0   
15600   19818      4071                    Dandenong Station      3175.0   
15452   19649      4088                     Box Hill Station      3128.0   
15444   19641      4113                     Box Hill Station      3128.0   
15769   19990      4318                          Hurstbridge      3099.0   
25174   46697      4346             Williams Landing Station      3027.0   
16724   21318      4362                     Ringwood Station      3134.0   
15371   19549      4393                              King St      3000.

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
## decline 

Delta_R_decline     = dataset_decline.CardID_x - dataset_decline.CardID_y  
total_number =  Delta_R_decline.count()
print('total number', total_number)
Delta_R_decline_ratio = Delta_R_decline / np.sqrt(total_number) 

# adding measured ratio column to the dataframe 
dataset_decline["Delta_R_decline"] = Delta_R_decline_ratio 
print(dataset_decline)

 
for i in range(0,len(dataset_decline)):
   folium.Circle(
      location=[dataset_decline.iloc[i]['lat_x'], dataset_decline.iloc[i]['long_x']],
      popup=dataset_decline.iloc[i]['StopName_x'],
      radius=dataset_decline.iloc[i]['Delta_R_decline'],
      color='red',
      fill=True,
      fill_color='red'
   ).add_to(m2)
 

# Save it as html
m2.save('mymap_2015_2017_touchOn_All.html')


total number 92
       StopID  CardID_x             StopName_x  PostCode_x      lat_x  \
975      1170      3870          Hoddle Street      3067.0 -37.799978   
15447   19644      3879       Box Hill Station      3128.0 -37.820000   
14500   18474      3882           Highpoint SC      3032.0 -37.773189   
15593   19810      3890      Monash University      3168.0 -37.913905   
4730     5848      3946              Clarke St      3020.0 -37.784988   
16653   21227      4000     Cranbourne Station      3977.0 -38.100225   
917      1106      4033     Blessington Street      3182.0 -37.870986   
16694   21282      4069        Croydon Station      3136.0 -37.795823   
2192     2707      4088  Greensborough Station      3088.0 -37.703429   
14752   18752      4113       Sunshine Station      3020.0 -37.787130   
7891     9663      4130        Holmesglen TAFE      3148.0 -37.875262   
15596   19813      4146      Monash University      3168.0 -37.914436   
16715   21309      4225    Waverley

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
